# Scikit Learn Pipelines

## Titanic Dataset wihout pipelines

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# this is a fast track project, just to demonstrate the lenghty process
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)
# removes columns

In [4]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [5]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [6]:
print(x_train.head(3))
print(y_train.head(3))

     Pclass   Sex   Age  SibSp  Parch    Fare Embarked
331       1  male  45.5      0      0  28.500        S
733       2  male  23.0      0      0  13.000        S
382       3  male  32.0      0      0   7.925        S
331    0
733    0
382    0
Name: Survived, dtype: int64


In [7]:
# check null values
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [8]:
# create objects
si_age = SimpleImputer() # default - fill with mean
si_embarked = SimpleImputer(strategy='most_frequent') # filled with most frequent value

# fit and transform both the test and train datasets
x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.transform(x_test[['Age']])
x_test_embarked = si_embarked.transform(x_test[['Embarked']])
# the outputs are numpy arrays

In [9]:
# One Hot Encode Categorical Column
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# this will create a non-sparse matrix while ignoring unknown values

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)
# since we already had a numpy array on which we handelled missing values

x_test_sex = ohe_sex.transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.transform(x_test_embarked)

We have transformed 'sex', 'age' and 'embarked' the rest are as is, we'll have to remove these columns from the original dataframe and concatenate with the trnasformed columns.

In [10]:
x_train_rem = x_train.drop(columns=['Sex', 'Age', 'Embarked'])
x_test_rem = x_test.drop(columns=['Sex', 'Age', 'Embarked'])

In [11]:
# now concatenate
x_train_transformed = np.concatenate((x_train_rem, x_train_age, x_train_sex, x_train_embarked), axis=1)
x_test_transformed = np.concatenate((x_test_rem, x_test_age, x_test_sex, x_test_embarked), axis=1)

In [12]:
x_train_transformed.shape

(712, 10)

In [13]:
# now train the model
# we are using Decision Trees
clf = DecisionTreeClassifier()
clf.fit(x_train_transformed, y_train)

DecisionTreeClassifier()

In [14]:
# predict from the model
y_pred = clf.predict(x_test_transformed)

In [15]:
# metrics
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)
# we have some accuracy....

0.776536312849162

In [16]:
# this is used to deploy the model you trained
import pickle

In [17]:
# export the model to a file location...
# here we have to export the objects for missing values and encoders as well
# since the new data is in non-encoded form
pickle.dump(ohe_sex, open('models/ohe_sex.pkl', 'wb'))
pickle.dump(ohe_embarked, open('models/ohe_embarked.pkl', 'wb'))
pickle.dump(clf, open('models/clf.pkl', 'wb'))

# open() is for the directory location relative from this file....
# 'wb' means "Write-Binary"
## Note: you'll have to create the folder 'models' beforehand

Refer, "predict_without_pipepline" to see deployment